In [1]:
# Importing the required modules.
from sklearn import datasets
import pandas as pd
import numpy as np
import math
import pydotplus
from collections import deque

In [2]:
#Loading the datset.
iris_ds=datasets.load_iris()

#creating data frame from the iris.
df=pd.DataFrame(iris_ds.data)

#labeling the columns.
df.columns=['sl','sw','pl','pw']
df.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
#Labeling the output.
def label(value,*arguments):
     
    ''' if MIN <= value < (MIN+MEAN)/2 then output is a.
        if (MIN+MEAN)/2 <= value < (MEAN) then output is b.
        if (MEAN) <= value < (MAX+MEAN)/2 then output is c.
        if (MAX+MEAN)/2 <= value < MAX the output is d.'''
    
    if value<arguments[0]:
        return 'a'
    elif value < arguments[1]:
        return 'b'
    elif value < arguments[2]:
        return 'c'
    else:
        return 'd'
        

In [4]:
def getLabel(df,column_name):
    
    #calculating the minimum value for column.
    minimum=df[column_name].min()
    
    #calculating the mean for column.
    second=df[column_name].mean()
    
    #calculating the (min+mean)/2.
    first=(minimum+second)/2
    
    #calculating the maximum value for column.
    maximum=df[column_name].max()
    
    #calculaing the (max+mean)/2 for column.
    third=(second+maximum)/2
    
    #applying label to the values of column.
    return df[column_name].apply(label,args=(first,second,third))

In [5]:
#Creating the columns corrosponding to the values present in initial column.

#labeling the values of column 'sl'.
df['sl_label']=getLabel(df,'sl')

#labeling the values of column 'sw'.
df['sw_label']=getLabel(df,'sw')

#labeling the values of column 'pl'.
df['pl_label']=getLabel(df,'pl')

#labeling the values of column 'pw'.
df['pw_label']=getLabel(df,'pw')
df.head()

,sl,sw,pl,pw,sl_label,sw_label,pl_label,pw_label
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a


In [6]:
#dropping the initial columns, as we got new columns with labeled output of initial column values.
df.drop(['sl','sw','pl','pw'],axis=1,inplace=True)
df.head()

,sl_label,sw_label,pl_label,pw_label
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [7]:
#Unique label for column 'pw_label.'
set(df['pw_label'])

{'a', 'b', 'c', 'd'}

In [8]:
class TreeNode:
    def __init__(self,data,output):
        
        #data contains the feature upon which the split is made while fitting the training data.
        self.data=data
        
        #output represents the class with maximum majority as this level.
        self.output=output
        
        #storing the childrens of node in key value pair key=feature and value contains child tree node.
        self.children={}
        
        #assigning particular index to node.
        self.index=-1
        
    def addChild(self,feature_val,object):
        
        #Method to add the children of a node.
        self.children[feature_val]=object

In [9]:
class DecisionTreeClassifier:
    def __init__(self):
        
        #Root of decison Tree.
        self.root=None
        
    def __frequencyCounter(self,y):
        '''
        Purpose:
            This method return the dictionary of diffrent labels present in y.
            key=label
            value=count of label.
        parameter:
            y:Target class
        '''
        #Creating dictionary to store labels with their count.
        di={}
        
        #itertaing over the target class.
        for i in y:
            
            #if label is previously present incresing its count.
            if i in di:
                di[i]+=1
            
            #if label occured first time the value for that is 1.
            else:
                di[i]=1
        return di
    
    def __entropy(self,y):
        '''
        Purpose:
            This method returns the entropy of particular node.
        Paramter:
        y:Target class
        '''
        
        #getting the dictionary of different labels and their frequency in target class.
        freq_lab=self.__frequencyCounter(y)
        
        #Assuming initial entropy as 0.
        ent=0
        
        #getting the total length of target class.
        total=len(y)
        
        #calculating the entropy.
        for i in freq_lab:
            p=(freq_lab[i]/total)
            ent+=(-p)*math.log2(p)
        
        return ent
    
    def __gainRatio(self,x,y,feature_index):
        '''Purpose:
            It returns the gain ratio.
            Parameters:
            x: Training data
            y: Target data
            feature_index: feature whose gain ratio to be calculated.
            '''
        #getting the entropy before splitting the node.
        entropy_bsplit=self.__entropy(y)
        
        #Assuming initial entropy after split to be 0. 
        entropy_asplit=0
        
        #Assuming split_info to be 0.
        split_info=0
        
        #getting the different values of selected feature.
        dif_val=set(x[:,feature_index])
        df=pd.DataFrame(x)
        
        #Appending the target value as last column of newly created dataframe.
        df[df.shape[1]]=y 
        org_=df.shape[0]
        
        #Calculating the entropy after split and split info.
        for i in dif_val:
            df1=df[df[feature_index]==i]
            new_=df1.shape[0]
            entropy_asplit+=(new_/org_)*self.__entropy(df1[df1.shape[1]-1])
            split_info+=(-new_/org_)*math.log2(new_/org_)
        
        #in case the split info is 0 division with it would let ZeroDivisonError, avoiding that and returning -infinty.
        if split_info==0:
            return -math.inf
        
        #Getting the information gain.
        info_gain=entropy_bsplit-entropy_asplit
        
        #calculating the gain ratio.
        gain_ratio=info_gain/split_info
        return gain_ratio
    
    def __decisonTree(self,x,y,features,level,classes, all_features=np.array([i for i in df.columns])):
        '''Purpose:
            Main Function for construction of decision Tree.
            Parameters:
            x            :Training data
            y            :Target value
            features     : List containing index of features present.
            level        : Current level
            classes      : differenet classes or labels present in Target.
            all_features : Names of different features available. 
        '''
        ############################################################
        # BASE CASE I : In case no features are left to split upon.#
        ############################################################
        
        # If no features are left then answer is the class with maximum value for given split.
        
        # Things to be done in BASE CASE I:
            # print the current level.
            # print the frequency of classes in case the class is not present in target then its value is zero, if present then its corrsoponding frequency.
            # Entrpoy.
            # Printing Reached leaf node.
        if len(features) == 0:
            
            # printing current level.
            print("Level",level)
            
            #getting the frequency of classes present in y.
            freqs = self.__frequencyCounter(y)
            output = None
            max_count = -99999999999999
            
            #printing the frequency of classes present at current level. 
            for i in classes:
                if i not in freqs:
                    print("Count of",i,"=",0)
                else :
                    if freqs[i] > max_count :
                        output = i
                        max_count = freqs[i]
                    print("Count of",i,"=",freqs[i])
            
            #printing the entropy at current level.
            print("Current Entropy  is =",self.__entropy(y))          
            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        #####################################################################
        # BASE CASE II : If there is value for single class i.e, pure class.#
        #####################################################################
        
        # If the node consists of a single pure class then no further split is possible.
        
        # Things to be done in BASE CASE II:
            # print the current level.
            # print the frequency of classes in case the class is not present in target then its value is zero, if present then its value would be len(set(y)).
            # Entrpoy is 0.0.
            # Printing Reached leaf node.
            
        if len(set(y)) == 1:
            
            #printing the level.
            print("Level",level)
            output = None
            
            #printing the value for the class.
            for class_ in classes:
                if class_ in y:
                    output = class_
                    print("Count of",class_,"=",len(y))
                else :
                    print("Count of",class_,"=",0)
                    
            #printing the current entropy.
            print("Current Entropy is =  0.0")
            print("Reached leaf Node")
            print()
            return TreeNode(None,output)
        
        #################################################################################################
        # Steps performed to get the best feature on the basis of GAIN RATIO on which split can be made.#                                                                                   
        #################################################################################################
        
        #Assuming the initail max_gain to be -infinity.
        max_gain = -math.inf
        
        #Assuming initial feature is None.
        final_feature = None
        
        #calculating gain ratio for the features and getting the feature with max gain ratio.
        for f in features :
            current_gain = self.__gainRatio(x,y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f
        
        # printing the current level.
        print("Level",level)
        freqs = self.__frequencyCounter(y)
        
        output = None
        max_count = -math.inf
        
        #printing the values for classes at current level.
        for class_ in classes:
            if class_ not in freqs:
                print("Count of",class_,"=",0)
            else :
                if freqs[class_] > max_count :
                    output = class_
                    max_count = freqs[class_]
                print("Count of",class_,"=",freqs[class_])
        
        #calculating and printing the entropy.
        print("Current Entropy is =",self.__entropy(y))
        
        #printing the name of feature upon which split is made with its gain ratio.
        print("Splitting on feature" , all_features[final_feature] , "with gain ratio" , max_gain)
        print()
        
        # unique_values represents the unique values of the feature selected    
        unique_values = set(x[:,final_feature]) 
        df = pd.DataFrame(x)
        
        # appending y values at the end of the data frame.
        df[df.shape[1]] = y

        current_node = TreeNode(final_feature,output)
        index  = features.index(final_feature)
        
        # removing the selected feature from the list
        features.remove(final_feature)
        
        # Creating the nodes for each unique values for the selected feature and calling recursion.
        for i in unique_values:
            
            # Creating a new dataframe with value of selected feature = i
            df_new = df[df[final_feature] == i]
            
            # recursively calling on the splits
            node = self.__decisonTree(df_new.iloc[:,0:df_new.shape[1]-1].values,df_new.iloc[:,df_new.shape[1]-1].values,features,level+1,classes)
            current_node.addChild(i,node)

        # Add the removed feature     
        features.insert(index,final_feature)

        return current_node
        
    
    def fit(self,x,y):
        
        '''Purpose: Fits according to the trainig data.
            Parameters: 
            x:Training data
            y:Target value. 
        '''
        #creating index array for features present in Training data.
        features=[i for i in range(len(x[0]))]
        
        #different classes in Target value.
        classes=set(y)
        
        #current level.
        level=0
        self.root=self.__decisonTree(x,y,features,level,classes)
        return self.root
    
    def __predict_single(self,data,node):
        '''Purpose: Predicts class for given testing point.
           Parameters:
           data: Feature name
           node: particular node.
        '''
        # In case we have reached a leaf node.
        if len(node.children) == 0 :
            return node.output
        
        # represents the value of feature on which the split is made.
        val = data[node.data]       
        if val not in node.children :
            return node.output
        
        # Recursively call on the splits.
        return self.__predict_single(data,node.children[val])
    
    def predict(self,x):
        '''Purpose: This function returns y_predicted.
           Parameter: 
           x: 2-D Numpy array.
        '''
        y=np.array([0 for i in range(len(x))])
        for i in range(len(x)):
            y[i]= self.__predict_single(x[i],self.root)
        return y
    
    def score(self,x,y):
        '''Purpose: To get the mean accuracy.
           Parameters : 
           x: Testing data.
           y: Target value.
        '''
        #getting the y_pred for given data.
        Y_predicted = self.predict(x)
        counter = 0
        for i in range(len(Y_predicted)):
            if Y_predicted[i] == y[i]:
                counter+=1
        return counter/len(Y_predicted)

In [10]:
from sklearn.model_selection import train_test_split

#Getting data as x
x=df.values

#Getting target value as y.
y = iris_ds.target

#splitting the data in training and testing.
x_train, x_test, y_train, y_test=train_test_split(x, y)
unused_features = set(df.columns)

#Creating object of DecisionTreeClassifier algorithm.
clf=DecisionTreeClassifier()

#Getting the root node.
root=clf.fit(x_train, y_train)
print()

#printing the score.
print("Score=", clf.score(x_test, y_test))

Level 0
Count of 0 = 41
Count of 1 = 34
Count of 2 = 37
Current Entropy is = 1.58071971384221
Splitting on feature pl_label with gain ratio 0.7160984564051073

Level 1
Count of 0 = 0
Count of 1 = 4
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 22
Current Entropy is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 30
Count of 2 = 15
Current Entropy is = 0.9182958340544896
Splitting on feature pw_label with gain ratio 0.37634368827214026

Level 2
Count of 0 = 0
Count of 1 = 4
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 9
Current Entropy is =  0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 26
Count of 2 = 6
Current Entropy is = 0.6962122601251458
Splitting on feature sl_label with gain ratio 0.11155079952800083

Level 3
Count of 0 = 0
Count of 1 = 9
Count of 2 = 0
Current Entropy is =  0.0
Reached leaf Node

Level 3
Count of 0 = 0
Cou

### Printing actual Decision Tree for Training data.

In [11]:
#Importing the required libraries.
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

C:\Users\pawan\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


### Creating dot_data file.

In [12]:
#Function below implemented returns tree as dot data.
def getTree_pdf(root):
    dot_data = '''digraph Tree {
node [shape=box] ;'''
    queue=deque()
    r=root
    queue.append(r)
    count=0
    if r.index==-1:
        r.index=count
    dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.data,r.output)    
    # Doing LEVEL ORDER traversal in the tree (using a queue)
    while len(queue)!=0:
        node=queue.popleft()
        for i in node.children:
            count+=1
            if node.children[i].index==-1:
                node.children[i].index=count
            
            # Creating child node.
            dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].data,node.children[i].output) 
            
            # Connecting parent node with child.
            dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
            
            # Adding child node to queue.
            queue.append(node.children[i])
    dot_data = dot_data + "\n}"
    return dot_data

In [13]:
dot_data = getTree_pdf(root)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.set_size('"10000,50!"')
Image(graph.create_png())